Importing required Libraries

In [236]:
import numpy as np
import cv2
import os
from tqdm import tqdm
from sklearn import linear_model, datasets
import pandas as pd
import time

Assigning paths

In [237]:
JOG_DIR='C:/Users/Sumanth_47/Jupyter/sampleo/jog_walk/jogging'
WALK_DIR='C:/Users/Sumanth_47/Jupyter/sampleo/jog_walk/walking'

Function to create the training data set:
    
    1)Collect all the files in Jogging directory and caluclate 'parameter' for each of them and load them into a numpy array
    
    2)Collect all the files in Walking directory and caluclate 'parameter' for each of them and append them to the numpyarray
    
    3)Return the final Numpy array
    
!! Run find_para() function before running this cell

In [238]:
def create_train_data():
    training_data =[]
    for vid in tqdm(os.listdir(JOG_DIR)):
        _type='jog'
        path1='jogging/'+vid
        frame,time =find_para(path1)
        
        training_data.append([vid,_type,frame,time])
        
    
    for vid in tqdm(os.listdir(WALK_DIR)):
        _type='walk'
        path2='walking/'+vid
        frame,time=find_para(path2)
        
        training_data.append([vid,_type,frame,time])
        
    
    
    
    return training_data

Calculating the Paramenters: 

Parameters are 1)the number of frames an object is visible for 2) time duration an object is visible for. It is found according to the following steps:


   1) Read the video file using OpenCV 
   
   2) Note the frame and the time stamp when the object is detected for the first time
   
   3) Note the frame and the time stamp when the object disappears for the first time
   
   4) Caluclate the difference between them to obtain both the paramenters

In [239]:
def find_para(video):
    
    cap = cv2.VideoCapture('C:/Users/Sumanth_47/Jupyter/sampleo/jog_walk/'+video)
    body_cascade=cv2.CascadeClassifier('sampleo/cascadG.xml')

    frame_counter = 0
    start_frame=0
    end_frame=0
    count=0
    start_time=0
    end_time=0


    while(cap.isOpened()):
        frame_counter=frame_counter+1
        ret, frame = cap.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gray = cv2.resize(gray, (700, 500))
    
   
    
        bodies=body_cascade.detectMultiScale(gray,1.3,5)
        for(x,y,w,h) in bodies:
        
            cv2.rectangle(gray,(x,y),(x+w,y+h),(255,0,0),2)
        
            if(x>=200 and start_frame==0 and count<2):
            
                start_frame = frame_counter
                start_time=time.time()
                count=count+1
            #print('sf=',start_frame,'|count:',count)
        
            elif(x<200 and count < 2 and start_frame!= 0):
                end_frame= frame_counter
                end_time=time.time()
                count=count+1 
    
    
    
    #cv2.imshow('gray',gray)
    
    
    
        if frame_counter == cap.get(cv2.CAP_PROP_FRAME_COUNT):
             break
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    

    cap.release()
    cv2.destroyAllWindows()

    total_frame=end_frame-start_frame
    total_time=end_time-start_time

    return total_frame,total_time


Create the Training data

In [240]:
t_d=create_train_data()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52/52 [04:43<00:00,  5.46s/it]


Convert the Numpy array to Data frame to Logisitic Regression (Classifier) from SKlearn and find the class of new video input 

In [248]:
df = pd.DataFrame(t_d)

dic={'jog':1, 'walk':0}

df[1]=df[1].apply(lambda x:dic[x])


Splitting Features and Labels

In [249]:
Y=df[1]
X=df[[2,3]]

Initializing Logistic Regression Classifier and fitting it

In [250]:
logreg = linear_model.LogisticRegression()
logreg.fit(X,Y)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [251]:
logreg.predict(X)

array([1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0], dtype=int64)